In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype='int')
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
indep=dataset[['Gender_Male','Age','EstimatedSalary']]
dep=dataset[['Purchased']]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy'],'max_features':['auto','sqrt','log2'],'n_estimators':[10,50,100]}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_pa

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 50, 100]},
             scoring='f1_weighted', verbose=3)

In [8]:
result=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [9]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print('confusion matrix:\n',cm)

confusion matrix:
 [[73  6]
 [ 4 37]]


In [10]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print('The clf report :\n',clf_report)

The clf report :
               precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print('The f1_macro value for the best parameters {}:'.format(grid.best_params_),f1_macro)

The f1_macro value for the best parameters {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 50}: 0.9171245421245421


In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9590923124421118

In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
1,0.000328,0.000655,0.000000,0.000000,gini,auto,50,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
2,0.000934,0.001867,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
3,0.043660,0.007641,0.016394,0.003994,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.855314,0.821429,0.841398,0.892857,0.888158,0.859831,0.027306,12
4,0.136719,0.007353,0.013996,0.005946,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.819142,0.823129,0.911105,0.946153,0.874757,0.049350,8
5,0.235978,0.011148,0.022722,0.002557,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.858503,0.841398,0.911105,0.964286,0.889909,0.043740,2
6,0.031096,0.000534,0.011526,0.002247,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.835985,0.857143,0.859435,0.911105,0.868182,0.866370,0.024743,11
7,0.125398,0.005284,0.017412,0.004101,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.838326,0.823129,0.876841,0.982051,0.878920,0.055538,5
8,0.246467,0.008296,0.020007,0.002250,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.857143,0.823129,0.911105,0.927778,0.878682,0.037500,6
9,0.001043,0.002086,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


In [14]:
import pickle

In [15]:
filename='Final_model_on_RF_clf.sav'
pickle.dump(grid,open(filename,'wb'))
loaded_model=pickle.load(open('Final_model_on_RF_clf.sav','rb'))
result=loaded_model.predict([[0,30,2000]])
result

array([1], dtype=int64)